In [12]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# [Problem 1] Learning / Estimation

In [13]:
image_folder = "D:/competition_data/competition_data/train/images"
mask_folder = "D:/competition_data/competition_data/train/masks"

image_files = os.listdir(image_folder)
mask_files = os.listdir(mask_folder)


In [14]:
IMG_HEIGHT = 128
IMG_WIDTH = 128

def preprocess_image(img, target_size=(IMG_HEIGHT, IMG_WIDTH)):
  img = cv2.resize(img, target_size)
  img = img / 255.0  
  return img


def load_images_and_masks(image_folder, mask_folder, image_files):
  images = []
  masks = []
  
  for image_file in image_files:
    image = cv2.imread(os.path.join(image_folder, image_file), cv2.IMREAD_GRAYSCALE)
    image = preprocess_image(image)

  
    mask = cv2.imread(os.path.join(mask_folder, image_file), cv2.IMREAD_GRAYSCALE)
    mask = preprocess_image(mask)

    images.append(image)
    masks.append(mask)
  
  return np.array(images), np.array(masks)

images, masks = load_images_and_masks(image_folder, mask_folder, image_files)


In [15]:
datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.2, horizontal_flip=True)

In [16]:
def unet_model(input_size=(128, 128, 1)):
  inputs = layers.Input(input_size)
  
  c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
  c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
  p1 = layers.MaxPooling2D((2, 2))(c1)

  c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
  c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
  p2 = layers.MaxPooling2D((2, 2))(c2)

  c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
  c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

  u2 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c3)
  u2 = layers.concatenate([u2, c2])
  c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u2)
  c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

  u1 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)
  u1 = layers.concatenate([u1, c1])
  c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
  c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

  outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c5)
  
  model = models.Model(inputs=[inputs], outputs=[outputs])
  return model

model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [18]:
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 863s 9s/step - accuracy: 0.7271 - loss: 0.5815 - val_accuracy: 0.7524 - val_loss: 0.5507
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1043s 10s/step - accuracy: 0.7444 - loss: 0.5584 - val_accuracy: 0.7524 - val_loss: 0.4663
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1024s 10s/step - accuracy: 0.7592 - loss: 0.4712 - val_accuracy: 0.8196 - val_loss: 0.4470
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1426s 14s/step - accuracy: 0.8131 - loss: 0.4343 - val_accuracy: 0.8446 - val_loss: 0.3787
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1012s 10s/step - accuracy: 0.8275 - loss: 0.3994 - val_accuracy: 0.8522 - val_loss: 0.3551
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1041s 10s/step - accuracy: 0.8423 - loss: 0.3743 - val_accuracy: 0.8708 - val_loss: 0.3215
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 980s 10s/step - accuracy: 0.8659 - loss: 0.3363 - val_accuracy: 0.8781 - val_loss: 0.2980
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1168s 12s/step - accuracy: 0.8646 - loss: 0.33

In [19]:
model.evaluate(X_val, y_val)

25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.9264 - loss: 0.1910


[0.20022931694984436, 0.9230294227600098]

# [Problem 2] Code reading

Le code fourni est destiné à la segmentation d'images, typiquement utilisée dans des tâches comme la détection d'objets ou la segmentation sémantique pour des classes multiples ou binaires. Voici une explication des différentes parties du code :

1- data.py
  Les premières lignes définissent différentes classes d'objets (ciel, bâtiment, poteau, route, etc.) représentées par des valeurs de couleur spécifiques en tant que listes RGB. Ces couleurs sont stockées dans un tableau numpy (`COLOR_DICT`).

  La fonction `adjustData` prend en entrée une image et un masque (qui représente les annotations des classes sur l'image), et effectue une normalisation ou une binarisation selon le cas 

  La fonction `trainGenerator` génère par lots des images et leurs masques associés pour l'entraînement du modèle, en appliquant des augmentations de données (rotations, inversions, etc.) sur les images et les masques de manière synchronisée pour garantir que les transformations sont identiques sur les deux. 

  La fonction `testGenerator` génère des images à partir du répertoire de test. Elle charge une image, la redimensionne et la normalise avant de la transformer en un format compatible avec le modèle (ajoute des dimensions si nécessaire)

  la fonction `geneTrainNpy` permet de charger un ensemble d'images et de masques à partir de fichiers `.png`, puis de les ajuster en utilisant la fonction `adjustData`. Elle retourne deux tableaux Numpy : un pour les images et un pour les masques

  La fonction `labelVisualize` convertit le masque de prédiction, qui contient des valeurs de classe, en une image RGB correspondant aux couleurs définies dans `COLOR_DICT`. Pour chaque classe, la fonction applique la couleur appropriée à tous les pixels appartenant à cette classe.

  la fonction `saveResult` enregistre les résultats des prédictions générées par le modèle sous forme d'images. Elle prend en entrée un tableau contenant les masques prédits et les convertit en images visuelles (RGB) à l'aide de `labelVisualize` si plusieurs classes sont présentes. Les résultats sont ensuite sauvegardés en tant que fichiers `.png`.

2- main.py 

  Les données d'entraînement sont augmentées à l'aide de transformations aléatoires.

  Un modèle U-Net est créé et entraîné sur ces données augmentées.

  Les images de test sont chargées, et le modèle effectue des prédictions sur celles-ci.

  Les prédictions sont sauvegardées sous forme d'images segmentées.

3- model.py
  Le modèle U-Net implémenté dans la fonction unet est un réseau convolutif pour la segmentation d'images, composé de :
  Un chemin descendant (encodeur) pour extraire des caractéristiques globales en réduisant progressivement la résolution de l'image.
  Un chemin montant (décodeur) pour restaurer la résolution originale tout en combinant les caractéristiques globales et locales grâce aux connexions entre les couches correspondantes de l'encodeur et du décodeur.
  Le modèle est compilé avec l'optimiseur Adam et est conçu pour des tâches de segmentation binaire avec une fonction de perte adaptée (binary_crossentropy).